In [ ]:
# 정해진 예시에 따라 프롬프트 양식을 구현할 수 있음
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
    temperature=1, 
    model_name = "gpt-3.5-turbo",
    streaming=True, 
    callbacks=[StreamingStdOutCallbackHandler(),],)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
] 

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}")
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography and curture expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Korea"})



In [ ]:
# 예제를 선택하는 기능 : BaseExampleSelector
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate 
from langchain.prompts.example_selector.base import BaseExampleSelector 


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")


example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")


In [30]:
# Serialization and Composition
# proompt json 파일과 yaml 파일을 생성하고 load_prompt를 통해서 연결할 수 있음 
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate # 많은 프롬프트를 합칠 수 있음

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
    {example}
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "character": "Pirate",
  "example_question": "What is your location?",
  "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
  "question": "What is your fav food?"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:PipelinePromptTemplate] Entering Prompt run with input:
{
  "character": "Pirate",
  "example_question": "What is your location?",
  "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
  "question": "What is your fav food?"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:PipelinePromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "base",
    "StringPromptValue"
  ],
  "kwargs": {
    "text": "\n    \n    You are a role playing assistant.\n    And you are impersonating a Pirate\n\n    \n    This is an example of how you talk:\n\n    Human: What is your location?\n    You: Arrrrg! That is a secret!! Arg 

AIMessageChunk(content="Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothing beats the taste o' the sea! Arrr!")

In [ ]:
# Caching 에 관하여! - 1  + debug + SQLiteCache를 통한 db 생성으로 답변 가져옴
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make Korean pasta")

In [ ]:
# Caching 에 관하여! - 2
chat.predict("How do you make Korean pasta")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)

with get_openai_callback() as usage:
    a = chat.predict("소주를 만드는 방법은?")
    b = chat.predict("맥주를 만드는 방법은?")
    print(a,b, "\n")
    print(usage.total_cost)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI

chat = ChatOpenAI(
    temperature = 0.1,
    model = "gpt-3.5-turbo",
    max_tokens = 400,
)

chat.save("model.jason")
